In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import datetime

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import metrics

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission_V9Inaty.csv')

In [6]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [7]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [8]:
sample.tail()

,Purchase,User_ID,Product_ID
233594,100,1006036,P00118942
233595,100,1006036,P00254642
233596,100,1006036,P00031842
233597,100,1006037,P00124742
233598,100,1006039,P00316642


In [9]:
train.columns.difference(test.columns)

Index(['Purchase'], dtype='object')

In [10]:
train.shape, test.shape

((550068, 12), (233599, 11))

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     233599 non-null  int64  
 1   Product_ID                  233599 non-null  object 
 2   Gender                      233599 non-null  object 
 3   Age                         233599 non-null  object 
 4   Occupation                  233599 non-null  int64  
 5   City_Category               233599 non-null  object 
 6   Stay_In_Current_City_Years  233599 non-null  object 
 7   Marital_Status              233599 non-null  int64  
 8   Product_Category_1          233599 non-null  int64  
 9   Product_Category_2          161255 non-null  float64
 10  Product_Category_3          71037 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


In [13]:
train.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

In [14]:
train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,376430.000000,166821.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,9.842329,12.668243,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,5.086590,4.125338,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,2.000000,3.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [15]:
train['source'] = 'train'
test['source'] = 'test'

In [17]:
df = pd.concat([train, test])

In [18]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370.0,train
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200.0,train
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422.0,train
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057.0,train
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969.0,train


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 233598
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     783667 non-null  object 
 1   Product_ID                  783667 non-null  object 
 2   Gender                      783667 non-null  object 
 3   Age                         783667 non-null  object 
 4   Occupation                  783667 non-null  int64  
 5   City_Category               783667 non-null  object 
 6   Stay_In_Current_City_Years  783667 non-null  object 
 7   Marital_Status              783667 non-null  int64  
 8   Product_Category_1          783667 non-null  int64  
 9   Product_Category_2          537685 non-null  float64
 10  Product_Category_3          237858 non-null  float64
 11  Purchase                    550068 non-null  float64
 12  source                      783667 non-null  object 
dtypes: float64(3),

In [20]:
df.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            245982
Product_Category_3            545809
Purchase                      233599
source                             0
dtype: int64

In [35]:
df['User_ID'].nunique()

5891

In [32]:
df['User_ID'].iloc[0]

1000001

In [ ]:
df['User_ID'].loc[0]

In [39]:
df['User_ID'] = df['User_ID'].astype('str')

In [42]:
#New column User_ID_4
df['User_ID_4'] = df['User_ID'].apply(lambda x : x[0:4])

In [41]:
del df['User_ID_4']

In [43]:
df['User_ID_4'].nunique()

7

In [44]:
df['User_ID_4'].unique()

array(['1000', '1001', '1002', '1003', '1004', '1005', '1006'],
      dtype=object)

In [49]:
df.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,User_ID_4
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370.0,train,1000
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200.0,train,1000
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422.0,train,1000
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057.0,train,1000
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969.0,train,1000
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227.0,train,1000
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215.0,train,1000
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854.0,train,1000
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686.0,train,1000
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871.0,train,1000


In [46]:
#Copy of original dataset, except User_id_4
df_copy1 = df.copy()

In [47]:
df['Product_ID'].nunique()

3677

In [50]:
#New column Product_ID_4
df['Product_ID_4'] = df['Product_ID'].apply(lambda x : x[0:4])

In [55]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,User_ID_4,Product_ID_4,Product_ID_5
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370.0,train,1000,P000,P0006
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200.0,train,1000,P002,P0024
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422.0,train,1000,P000,P0008
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057.0,train,1000,P000,P0008
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969.0,train,1000,P002,P0028


In [52]:
df['Product_ID_4'].nunique()

5

In [56]:
df['Product_ID_4'].unique()

array(['P000', 'P002', 'P001', 'P003', 'P009'], dtype=object)

In [53]:
#New column Product_ID_5
df['Product_ID_5'] = df['Product_ID'].apply(lambda x : x[0:5])

In [54]:
df['Product_ID_5'].nunique()

45

In [57]:
df['Product_ID_5'].unique()

array(['P0006', 'P0024', 'P0008', 'P0028', 'P0019', 'P0018', 'P0034',
       'P0097', 'P0027', 'P0025', 'P0001', 'P0003', 'P0014', 'P0023',
       'P0096', 'P0005', 'P0022', 'P0015', 'P0021', 'P0030', 'P0013',
       'P0016', 'P0007', 'P0011', 'P0029', 'P0026', 'P0010', 'P0094',
       'P0031', 'P0036', 'P0012', 'P0033', 'P0004', 'P0032', 'P0035',
       'P0098', 'P0002', 'P0000', 'P0017', 'P0020', 'P0099', 'P0009',
       'P0037', 'P0095', 'P0093'], dtype=object)

In [58]:
df['Gender'].nunique()

2

In [59]:
df['Age'].nunique()

7

In [60]:
df['Age'].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [63]:
df['Age'] = df['Age'].replace({'0-17':'17', '55+':'60', '26-35':'35', '46-50':'50', '51-55':'55', '36-45':'45','18-25':'25'})

In [64]:
df['Age'].unique()

array(['17', '60', '35', '50', '55', '45', '25'], dtype=object)

In [65]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,User_ID_4,Product_ID_4,Product_ID_5
0,1000001,P00069042,F,17,10,A,2,0,3,NaN,NaN,8370.0,train,1000,P000,P0006
1,1000001,P00248942,F,17,10,A,2,0,1,6.0,14.0,15200.0,train,1000,P002,P0024
2,1000001,P00087842,F,17,10,A,2,0,12,NaN,NaN,1422.0,train,1000,P000,P0008
3,1000001,P00085442,F,17,10,A,2,0,12,14.0,NaN,1057.0,train,1000,P000,P0008
4,1000002,P00285442,M,60,16,C,4+,0,8,NaN,NaN,7969.0,train,1000,P002,P0028


In [76]:
df.shape[1]

16

In [71]:
df['Occupation'].unique()

array([10, 16, 15,  7, 20,  9,  1, 12, 17,  0,  3,  4, 11,  8, 19,  2, 18,
        5, 14, 13,  6], dtype=int64)

In [78]:
df['Occupation'].value_counts()

4     103086
0      98850
7      84127
1      67687
17     57418
20     47840
12     44448
14     38782
2      37996
16     36122
6      29102
3      25126
10     18304
5      17373
15     17356
11     16593
19     11919
13     11053
18      9367
9       8929
8       2189
Name: Occupation, dtype: int64

In [77]:
df.groupby(['Age']).Occupation.value_counts()

Age  Occupation
17   10            15493
     0              2938
     19             1153
     1               544
     12              356
                   ...  
60   8               234
     11              227
     10              170
     18              144
     5                19
Name: Occupation, Length: 134, dtype: int64

In [81]:
df['City_Category'].unique()

array(['A', 'C', 'B'], dtype=object)

In [79]:
df.groupby(['City_Category']).Occupation.value_counts()

City_Category  Occupation
A              4             34577
               0             26874
               7             22956
               1             18200
               20            18070
                             ...  
C              5              4526
               18             4091
               19             4042
               9              3356
               8               877
Name: Occupation, Length: 63, dtype: int64

In [82]:
df['Stay_In_Current_City_Years'].unique()

array(['2', '4+', '3', '1', '0'], dtype=object)

In [83]:
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].replace({'4+':'5'})

In [84]:
df['Stay_In_Current_City_Years'].unique()

array(['2', '5', '3', '1', '0'], dtype=object)

In [85]:
df['Marital_Status'].value_counts()

0    462538
1    321129
Name: Marital_Status, dtype: int64

In [87]:
df.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            245982
Product_Category_3            545809
Purchase                      233599
source                             0
User_ID_4                          0
Product_ID_4                       0
Product_ID_5                       0
dtype: int64

In [86]:
df['Product_Category_1'].nunique()

20

In [88]:
df['Product_Category_1'].unique()

array([ 3,  1, 12,  8,  5,  4,  2,  6, 14, 11, 13, 15,  7, 16, 18, 10, 17,
        9, 20, 19], dtype=int64)

In [106]:
df['Product_Category_2'].nunique()

17

In [90]:
df['Product_Category_2'].unique()

array([nan,  6., 14.,  2.,  8., 15., 16., 11.,  5.,  3.,  4., 12.,  9.,
       10., 17., 13.,  7., 18.])

In [102]:
df.groupby(['Product_Category_1']).Product_Category_2.mean()

Product_Category_1
1      7.740964
2      6.659880
3      4.482397
4      6.005866
5     10.952786
6     10.046307
7     12.504673
8     14.651326
9     15.000000
10    14.364661
11    15.116092
12    14.333441
13    15.626527
14    16.324185
15    16.215521
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
Name: Product_Category_2, dtype: float64

In [108]:
df.Product_Category_2.mean()

9.844505612021909

In [113]:
df.loc[df['Product_Category_2'] == np.NAN, 'Product_Category_2'] = 0.00000

In [114]:
df.groupby(['Product_Category_1']).Product_Category_2.mean()

Product_Category_1
1      7.740964
2      6.659880
3      4.482397
4      6.005866
5     10.952786
6     10.046307
7     12.504673
8     14.651326
9     15.000000
10    14.364661
11    15.116092
12    14.333441
13    15.626527
14    16.324185
15    16.215521
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
Name: Product_Category_2, dtype: float64

In [105]:
df['Product_Category_2'].value_counts(ascending = False)

8.0     91317
14.0    78834
2.0     70498
16.0    61687
15.0    54114
5.0     37165
4.0     36705
6.0     23575
11.0    20230
17.0    19104
13.0    15054
9.0      8177
12.0     7801
10.0     4420
3.0      4123
18.0     4027
7.0       854
Name: Product_Category_2, dtype: int64

In [110]:
df.sort_values(by = ['Product_Category_2','Product_Category_3'], ascending=False)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,User_ID_4,Product_ID_4,Product_ID_5
1301,1000208,P00343042,M,45,17,C,1,1,5,18.0,NaN,6975.0,train,1000,P003,P0034
1302,1000208,P00326242,M,45,17,C,1,1,2,18.0,NaN,6447.0,train,1000,P003,P0032
2133,1000333,P00343042,M,45,2,A,0,1,5,18.0,NaN,5302.0,train,1000,P003,P0034
2199,1000343,P00041942,F,45,3,B,2,0,5,18.0,NaN,5305.0,train,1000,P000,P0004
2242,1000349,P00111242,M,17,10,C,0,0,1,18.0,NaN,15265.0,train,1000,P001,P0011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233576,1006003,P00313342,F,50,17,C,1,0,5,NaN,NaN,NaN,test,1006,P003,P0031
233585,1006025,P00161142,F,35,1,B,1,0,11,NaN,NaN,NaN,test,1006,P001,P0016
233590,1006032,P00252442,M,50,7,A,3,0,8,NaN,NaN,NaN,test,1006,P002,P0025
233593,1006036,P00255442,F,35,15,B,5,1,8,NaN,NaN,NaN,test,1006,P002,P0025


In [116]:
df['Product_Category_2'] = df.groupby(['Product_Category_1']).Product_Category_2.transform(lambda x: x.fillna(x.mean()))

In [117]:
df.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             23323
Product_Category_3            545809
Purchase                      233599
source                             0
User_ID_4                          0
Product_ID_4                       0
Product_ID_5                       0
dtype: int64

In [118]:
df.groupby(['Product_Category_1']).Product_Category_2.mean()

Product_Category_1
1      7.740964
2      6.659880
3      4.482397
4      6.005866
5     10.952786
6     10.046307
7     12.504673
8     14.651326
9     15.000000
10    14.364661
11    15.116092
12    14.333441
13    15.626527
14    16.324185
15    16.215521
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
Name: Product_Category_2, dtype: float64

In [128]:
df['Product_Category_2'] = df['Product_Category_2'].transform(lambda x: x.fillna(df['Product_Category_2'].mean()))

In [130]:
df.groupby(['Product_Category_1']).Product_Category_2.mean()

Product_Category_1
1      7.740964
2      6.659880
3      4.482397
4      6.005866
5     10.952786
6     10.046307
7     12.504673
8     14.651326
9     15.000000
10    14.364661
11    15.116092
12    14.333441
13    15.626527
14    16.324185
15    16.215521
16    10.700281
17    10.700281
18    10.700281
19    10.700281
20    10.700281
Name: Product_Category_2, dtype: float64

In [131]:
df.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2                 0
Product_Category_3            545809
Purchase                      233599
source                             0
User_ID_4                          0
Product_ID_4                       0
Product_ID_5                       0
dtype: int64

In [132]:
df['Product_Category_3'].nunique()

15

In [133]:
df['Product_Category_3'].unique()

array([nan, 14., 17.,  5.,  4., 16., 15.,  8.,  9., 13.,  6., 12.,  3.,
       18., 11., 10.])

In [134]:
df.groupby(['Product_Category_1','Product_Category_2']).Product_Category_3.mean()

Product_Category_1  Product_Category_2
1                   2.000000              11.214108
                    3.000000               4.000000
                    4.000000               9.000000
                    5.000000              11.467642
                    6.000000              11.677581
                                            ...    
16                  10.700281                   NaN
17                  10.700281                   NaN
18                  10.700281                   NaN
19                  10.700281                   NaN
20                  10.700281                   NaN
Name: Product_Category_3, Length: 106, dtype: float64

In [135]:
df.groupby(['Product_Category_2']).Product_Category_3.mean()

Product_Category_2
2.000000     11.214108
3.000000      8.108659
4.000000      8.098805
4.482397           NaN
5.000000     11.543348
6.000000     11.576054
6.005866           NaN
6.659880           NaN
7.000000           NaN
7.740964           NaN
8.000000     14.864945
9.000000     14.272019
10.000000    13.880405
10.046307          NaN
10.700281          NaN
10.952786          NaN
11.000000    14.400194
12.000000    14.487616
12.504673          NaN
13.000000    15.449160
14.000000    16.439657
14.333441          NaN
14.364661          NaN
14.651326          NaN
15.000000    16.239066
15.116092          NaN
15.626527          NaN
16.000000    17.000000
16.215521          NaN
16.324185          NaN
17.000000          NaN
18.000000          NaN
Name: Product_Category_3, dtype: float64

In [136]:
df.groupby(['Product_Category_1']).Product_Category_3.mean()

Product_Category_1
1     12.771366
2     10.581451
3      8.208315
4     10.631151
5     14.170338
6     14.132325
7           NaN
8     16.304804
9           NaN
10    16.000000
11    16.000000
12    17.000000
13    16.000000
14          NaN
15    17.000000
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
Name: Product_Category_3, dtype: float64

In [138]:
df['Product_Category_3'].mean()

12.668604797820548

In [139]:
df['Product_Category_3'] = df['Product_Category_3'].transform(lambda x: x.fillna(df['Product_Category_3'].mean()))

In [140]:
df.groupby(['Product_Category_1']).Product_Category_3.mean()

Product_Category_1
1     12.735309
2     11.249896
3      9.925908
4     11.741926
5     12.856768
6     13.265702
7     12.668605
8     12.956823
9     12.668605
10    13.813560
11    12.934130
12    12.843868
13    13.297915
14    12.668605
15    12.773226
16    12.668605
17    12.668605
18    12.668605
19    12.668605
20    12.668605
Name: Product_Category_3, dtype: float64

In [141]:
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,User_ID_4,Product_ID_4,Product_ID_5
0,1000001,P00069042,F,17,10,A,2,0,3,4.482397,12.668605,8370.0,train,1000,P000,P0006
1,1000001,P00248942,F,17,10,A,2,0,1,6.000000,14.000000,15200.0,train,1000,P002,P0024
2,1000001,P00087842,F,17,10,A,2,0,12,14.333441,12.668605,1422.0,train,1000,P000,P0008
3,1000001,P00085442,F,17,10,A,2,0,12,14.000000,12.668605,1057.0,train,1000,P000,P0008
4,1000002,P00285442,M,60,16,C,5,0,8,14.651326,12.668605,7969.0,train,1000,P002,P0028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1006036,P00118942,F,35,15,B,5,1,8,14.651326,12.668605,NaN,test,1006,P001,P0011
233595,1006036,P00254642,F,35,15,B,5,1,5,8.000000,12.668605,NaN,test,1006,P002,P0025
233596,1006036,P00031842,F,35,15,B,5,1,1,5.000000,12.000000,NaN,test,1006,P000,P0003
233597,1006037,P00124742,F,50,1,C,5,0,10,16.000000,12.668605,NaN,test,1006,P001,P0012


In [142]:
df.groupby(['Product_ID']).Purchase.value_counts()

Product_ID  Purchase
P00000142   10715.0     5
            10995.0     5
            13610.0     5
            13691.0     5
            10596.0     4
                       ..
P0099942    6928.0      1
            7046.0      1
            7120.0      1
            7176.0      1
            8833.0      1
Name: Purchase, Length: 473633, dtype: int64

In [143]:
df.groupby(['Product_ID_4']).Purchase.value_counts()

Product_ID_4  Purchase
P000          7795.0      60
              7961.0      56
              8128.0      56
              7995.0      55
              8001.0      55
                          ..
P009          20806.0      1
              20833.0      1
              20870.0      1
              20884.0      1
              21050.0      1
Name: Purchase, Length: 63018, dtype: int64

In [148]:
df['Product_Category_3'] = df['Product_Category_3'].transform(lambda x: x.fillna(df['Product_Category_3'].mean()))

In [149]:
df.groupby(['Product_Category_1']).Purchase.mean()

Product_Category_1
1     13606.218596
2     11251.935384
3     10096.705734
4      2329.659491
5      6240.088178
6     15838.478550
7     16365.689600
8      7498.958078
9     15537.375610
10    19675.570927
11     4685.268456
12     1350.859894
13      722.400613
14    13141.625739
15    14780.451828
16    14766.037037
17    10170.759516
18     2972.864320
19       37.041797
20      370.481176
Name: Purchase, dtype: float64

In [150]:
df.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,User_ID_4,Product_ID_4,Product_ID_5
0,1000001,P00069042,F,17,10,A,2,0,3,4.482397,12.668605,8370.0,train,1000,P000,P0006
1,1000001,P00248942,F,17,10,A,2,0,1,6.000000,14.000000,15200.0,train,1000,P002,P0024
2,1000001,P00087842,F,17,10,A,2,0,12,14.333441,12.668605,1422.0,train,1000,P000,P0008
3,1000001,P00085442,F,17,10,A,2,0,12,14.000000,12.668605,1057.0,train,1000,P000,P0008
4,1000002,P00285442,M,60,16,C,5,0,8,14.651326,12.668605,7969.0,train,1000,P002,P0028
5,1000003,P00193542,M,35,15,A,3,0,1,2.000000,12.668605,15227.0,train,1000,P001,P0019
6,1000004,P00184942,M,50,7,B,2,1,1,8.000000,17.000000,19215.0,train,1000,P001,P0018
7,1000004,P00346142,M,50,7,B,2,1,1,15.000000,12.668605,15854.0,train,1000,P003,P0034
8,1000004,P0097242,M,50,7,B,2,1,1,16.000000,12.668605,15686.0,train,1000,P009,P0097
9,1000005,P00274942,M,35,20,A,1,1,8,14.651326,12.668605,7871.0,train,1000,P002,P0027


In [183]:
df.corr()

,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
Occupation,1.000000,0.026427,0.023563,-0.008441,-0.005903,0.005725,0.020833
Stay_In_Current_City_Years,0.026427,1.000000,-0.012366,-0.001252,0.000092,0.000928,0.004692
Marital_Status,0.023563,-0.012366,1.000000,0.019579,0.017572,0.010148,-0.000463
Product_Category_1,-0.008441,-0.001252,0.019579,1.000000,0.522884,0.083383,-0.343703
Product_Category_2,-0.005903,0.000092,0.017572,0.522884,1.000000,0.294862,-0.241800
Product_Category_3,0.005725,0.000928,0.010148,0.083383,0.294862,1.000000,-0.012284
Purchase,0.020833,0.004692,-0.000463,-0.343703,-0.241800,-0.012284,1.000000


In [184]:
X = df.select_dtypes(include=np.number).drop(columns =['Purchase'])

In [185]:
X.head()

,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,10,2,0,3,4.482397,12.668605
1,10,2,0,1,6.000000,14.000000
2,10,2,0,12,14.333441,12.668605
3,10,2,0,12,14.000000,12.668605
4,16,5,0,8,14.651326,12.668605


In [186]:
X.columns

Index(['Occupation', 'Stay_In_Current_City_Years', 'Marital_Status',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3'],
      dtype='object')

In [187]:
VIF_data = pd.DataFrame()

In [188]:
VIF_data['feature'] = X.columns

In [189]:
VIF_data

,feature
0,Occupation
1,Stay_In_Current_City_Years
2,Marital_Status
3,Product_Category_1
4,Product_Category_2
5,Product_Category_3


In [190]:
VIF_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

In [191]:
VIF_data

,feature,VIF
0,Occupation,2.443980
1,Stay_In_Current_City_Years,2.539439
2,Marital_Status,1.670607
3,Product_Category_1,3.998329
4,Product_Category_2,9.741885
5,Product_Category_3,9.915637


In [163]:
del df['Product_ID_5']

In [182]:
df['Age'] = df['Age'].astype('str')
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].astype('int64')

In [192]:
df_number_columns = df.select_dtypes(include=np.number).columns
df_number_columns

Index(['Occupation', 'Stay_In_Current_City_Years', 'Marital_Status',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Purchase'],
      dtype='object')

In [193]:
df_category_columns = df.select_dtypes(exclude=np.number).columns
df_category_columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'City_Category', 'source',
       'User_ID_4', 'Product_ID_4'],
      dtype='object')

In [194]:
data_to_encode = df[df_category_columns].drop(columns = ['User_ID', 'Product_ID','source'])

In [195]:
data_to_encode

,Gender,Age,City_Category,User_ID_4,Product_ID_4
0,F,17,A,1000,P000
1,F,17,A,1000,P002
2,F,17,A,1000,P000
3,F,17,A,1000,P000
4,M,60,C,1000,P002
...,...,...,...,...,...
233594,F,35,B,1006,P001
233595,F,35,B,1006,P002
233596,F,35,B,1006,P000
233597,F,50,C,1006,P001


In [196]:
df_category_encoded =  pd.get_dummies(data_to_encode)
df_category_encoded

,Gender_F,Gender_M,Age_17,Age_25,Age_35,Age_45,Age_50,Age_55,Age_60,City_Category_A,...,User_ID_4_1002,User_ID_4_1003,User_ID_4_1004,User_ID_4_1005,User_ID_4_1006,Product_ID_4_P000,Product_ID_4_P001,Product_ID_4_P002,Product_ID_4_P003,Product_ID_4_P009
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
233595,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
233596,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
233597,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [198]:
df_final = pd.concat([df_category_encoded, df[df_number_columns], 
                      df[['User_ID', 'Product_ID','source']]], axis = 1)

In [199]:
df_final.head()

,Gender_F,Gender_M,Age_17,Age_25,Age_35,Age_45,Age_50,Age_55,Age_60,City_Category_A,...,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,User_ID,Product_ID,source
0,1,0,1,0,0,0,0,0,0,1,...,10,2,0,3,4.482397,12.668605,8370.0,1000001,P00069042,train
1,1,0,1,0,0,0,0,0,0,1,...,10,2,0,1,6.000000,14.000000,15200.0,1000001,P00248942,train
2,1,0,1,0,0,0,0,0,0,1,...,10,2,0,12,14.333441,12.668605,1422.0,1000001,P00087842,train
3,1,0,1,0,0,0,0,0,0,1,...,10,2,0,12,14.000000,12.668605,1057.0,1000001,P00085442,train
4,0,1,0,0,0,0,0,0,1,0,...,16,5,0,8,14.651326,12.668605,7969.0,1000002,P00285442,train


In [200]:
train_preprocessed = df_final[df_final['source'] == 'train']
test_preprocessed = df_final[df_final['source'] == 'test']

In [201]:
train_preprocessed.shape, test_preprocessed.shape

((550068, 34), (233599, 34))

In [202]:
del train_preprocessed['source']

In [203]:
del test_preprocessed['source']

In [207]:
train_x = train_preprocessed.drop(columns = ['User_ID', 'Product_ID', 'Purchase'])

In [209]:
test_x = test_preprocessed.drop(columns = ['User_ID', 'Product_ID', 'Purchase'])

In [210]:
train_y = train_preprocessed['Purchase']

In [211]:
train_y[0:5]

0     8370.0
1    15200.0
2     1422.0
3     1057.0
4     7969.0
Name: Purchase, dtype: float64

In [212]:
sample.head()

,Purchase,User_ID,Product_ID
0,100,1000004,P00128942
1,100,1000009,P00113442
2,100,1000010,P00288442
3,100,1000010,P00145342
4,100,1000011,P00053842


### Modeling and metrics

In [213]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [215]:
lm = LinearRegression()
lm.fit(train_x, train_y)
pred_train = lm.predict(train_x)
pred_test = lm.predict(test_x)
pred_test = abs(pred_test)
print('MAE: ', mean_absolute_error(train_y, pred_train))
print('RMSE: ', np.sqrt(mean_squared_error(train_y, pred_train)))
print('R-squared: ', r2_score(train_y, pred_train))

MAE:  3578.8932259284306
RMSE:  4658.533243263984
R-squared:  0.13987509448269153


In [216]:
sample['Purchase'] = pred_test

In [217]:
sample.head(10)

,Purchase,User_ID,Product_ID
0,11360.625,1000004,P00128942
1,11849.750,1000009,P00113442
2,8305.750,1000010,P00288442
3,9976.750,1000010,P00145342
4,10668.875,1000011,P00053842
5,11550.625,1000013,P00350442
6,12080.000,1000013,P00155442
7,10249.875,1000013,P0094542
8,7926.625,1000015,P00161842
9,9055.000,1000022,P00067942


In [221]:
test.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,source
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN,test
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN,test
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN,test
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN,test
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0,test
5,1000013,P00350442,M,46-50,1,C,3,1,2,3.0,15.0,test
6,1000013,P00155442,M,46-50,1,C,3,1,1,11.0,15.0,test
7,1000013,P0094542,M,46-50,1,C,3,1,2,4.0,9.0,test
8,1000015,P00161842,M,26-35,7,A,1,0,10,13.0,16.0,test
9,1000022,P00067942,M,18-25,15,A,4+,0,5,14.0,NaN,test


In [220]:
sample.shape, test.shape

((233599, 3), (233599, 12))

In [218]:
sample.to_csv('Black_friday_sales.csv')